In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./4.item_2.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests
                import json

                import sys
                sys.path.append('../../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.post(input_.loc[a, 'Vehicle Url'],
                                     data=input_.loc[a, 'Data'],
                                     headers=get_header(ua=False, **{'Content-Type': 'application/json; charset=utf-8'}),
                                     proxies=get_proxy(),
                                     timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp.strip())

                # = = = = = = = = = = = = = = =

                list_header = [cell['textSpans'][0]['text'].strip() for cell in json_['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']]

                # = = = = = = = = = = = = = = =

                list_row = [[cell['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in cell else cell['textSpans'][0]['text'].strip() for cell in row['cells']] for row in json_['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                        'Item Number': input_.loc[a, 'Item Number'],
                                        'Market': input_.loc[a, 'Market'],
                                        'Page': input_.loc[a, 'Page'],
                                        'JOIN_INP': input_.loc[a, 'JOIN_INP'],
                                        'Row': [i+1 for i in range(len(list_row))]})

                for i, row in zip(range(len(list_row)), list_row):
                    for j, header in zip(range(len(list_header)), list_header):
                        df_temp.loc[i, header] = row[j]

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Item Number': input_.loc[a, 'Item Number'],
                                     'Market': input_.loc[a, 'Market'],
                                     'Page': input_.loc[a, 'Page'],
                                     'JOIN_INP': input_.loc[a, 'JOIN_INP'],
                                     'Vehicle Url': input_.loc[a, 'Vehicle Url'],
                                     'Data': input_.loc[a, 'Data']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']} > {input_.loc[a, 'Page']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./5.vehicle_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No', 'Page'],
                                            ascending=[True, True],
                                            ignore_index=True)
    output_error.to_excel(f'./vehicle_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：  8] - 419.324302564509 > 6
[剩余数量：1000] - [当前时间：19:57:04]

[状态：ok，尝试次数： 80] - 417.224164566363 > 3
[剩余数量：999] - [当前时间：19:57:04]

[状态：ok，尝试次数：  1] - 419.324302564509 > 7
[剩余数量：998] - [当前时间：19:57:04]

[状态：ok，尝试次数：  4] - 419.324302564509 > 10
[剩余数量：997] - [当前时间：19:57:05]

[状态：ok，尝试次数：  3] - 419.324302564509 > 12
[剩余数量：996] - [当前时间：19:57:05]

[状态：ok，尝试次数：  6] - 419.324302564509 > 11
[剩余数量：995] - [当前时间：19:57:05]

[状态：ok，尝试次数：  1] - 420.324302564528 > 1
[剩余数量：994] - [当前时间：19:57:05]

[状态：ok，尝试次数：  2] - 420.324302564528 > 4
[剩余数量：993] - [当前时间：19:57:06]

[状态：ok，尝试次数： 29] - 419.324302564509 > 2
[剩余数量：992] - [当前时间：19:57:06]

[状态：ok，尝试次数：  2] - 420.324302564528 > 5
[剩余数量：991] - [当前时间：19:57:06]

[状态：ok，尝试次数： 11] - 420.324302564528 > 7
[剩余数量：990] - [当前时间：19:57:07]

[状态：ok，尝试次数： 18] - 420.324302564528 > 3
[剩余数量：989] - [当前时间：19:57:07]

[状态：ok，尝试次数：  4] - 420.324302564528 > 8
[剩余数量：988] - [当前时间：19:57:08]

[状态：ok，尝试次数： 14] - 420.324302564528 > 2
[剩余数量：987] - [当前时间：19:57:08]

[状态：ok，尝试次数： 12]